In [1]:
import requests

year = 2024

url = f"https://feriadosargentina.com.ar/feriados/{year}"
response = requests.get(url)
print(response)

<Response [200]>


In [2]:
response.text

'\n<!DOCTYPE html>\n\n<html lang="es">\n\n\n<!DOCTYPE html>\n<html lang="es">\n\n<head>\n  <title>Feriados Argentina 2024</title><meta name=\'description\' content=\'Calendario completo de feriados, feriados puentes y dias no laborables, mes por mes en Argentina 2024. \'/><meta name=\'keywords\' content=\'Feriados Nacionales, Boletín Oficial, Próximo feriado, Argentina, Feriados, feriados puente, feriados argentina.gob.ar, 2024\'/>\n  <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n  <meta name="viewport" content="width=device-width">\n\n\n  <base href=\'/\'>\n  <link rel="icon" type="image/x-icon" href="/images/favicon.ico">\n\n  <link rel="preconnect" href="https://pagead2.googlesyndication.com" crossorigin>\n  <link rel="preconnect" href="https://tpc.googlesyndication.com" crossorigin>\n  <link rel="preconnect" href="https://fonts.googleapis.com" crossorigin>\n  <link rel="preconnect" href="https://www.googletagmanager.com" crossorigin>\n  <link rel="preconnec

In [3]:
with open("response.txt", "w") as f:
  f.write(response.text)

In [4]:
import re

pattern = r"<td id='([^']+)' style='([^']+)'>"
result = re.findall(pattern, response.text)
list(result)

[('January-1-2024', 'background-color:#0072bb; color:#fff;'),
 ('February-12-2024', 'background-color:#0072bb; color:#fff;'),
 ('February-13-2024', 'background-color:#0072bb; color:#fff;'),
 ('March-24-2024', 'background-color:#0072bb; color:#fff;'),
 ('March-28-2024', 'background-color:#595959; color:#fff;'),
 ('March-29-2024', 'background-color:#0072bb; color:#fff;'),
 ('April-1-2024', 'background-color:#6a1b99; color:#fff;'),
 ('April-2-2024', 'background-color:#0072bb; color:#fff;'),
 ('April-10-2024', 'background-color:#595959; color:#fff;'),
 ('April-24-2024', 'background-color:#595959; color:#fff;'),
 ('May-1-2024', 'background-color:#0072bb; color:#fff;'),
 ('May-25-2024', 'background-color:#0072bb; color:#fff;'),
 ('June-17-2024', 'background-color:#2e7d33; color:#fff;'),
 ('June-20-2024', 'background-color:#0072bb; color:#fff;'),
 ('July-7-2024', 'background-color:#595959; color:#fff;'),
 ('July-9-2024', 'background-color:#0072bb; color:#fff;'),
 ('August-17-2024', 'backgroun

In [5]:
import pandas as pd

df = pd.DataFrame(result, columns=["raw_date", "style"])
df

,raw_date,style
0,January-1-2024,background-color:#0072bb; color:#fff;
1,February-12-2024,background-color:#0072bb; color:#fff;
2,February-13-2024,background-color:#0072bb; color:#fff;
3,March-24-2024,background-color:#0072bb; color:#fff;
4,March-28-2024,background-color:#595959; color:#fff;
5,March-29-2024,background-color:#0072bb; color:#fff;
6,April-1-2024,background-color:#6a1b99; color:#fff;
7,April-2-2024,background-color:#0072bb; color:#fff;
8,April-10-2024,background-color:#595959; color:#fff;
9,April-24-2024,background-color:#595959; color:#fff;


In [6]:
def get_tipo(style):
  pattern = f"<td style='{style} width: 50px;'></td>\s*<td class='text-center'>([^<]+)</td>"
  result = re.findall(pattern, response.text)
  return result[0] if len(result) != 0 else None

df["tipo"] = df["style"].apply(get_tipo)

In [7]:
a_eliminar = df[df["tipo"].isna()]
print(a_eliminar)
df.drop(index=a_eliminar.index, inplace=True)

Empty DataFrame
Columns: [raw_date, style, tipo]
Index: []


In [8]:
import calendar

list(calendar.month_name)

['',
 'January',
 'February',
 'March',
 'April',
 'May',
 'June',
 'July',
 'August',
 'September',
 'October',
 'November',
 'December']

In [9]:
from datetime import date

MONTHS = list(calendar.month_name)

def get_date(raw_date):
  pattern = "([^-]+)-([^-]+)-(\d+)"
  result = re.findall(pattern, raw_date)
  raw_month, raw_day, raw_year = result[0]
  month = MONTHS.index(raw_month)
  return date(int(raw_year), month, int(raw_day))

df["fecha"] = df["raw_date"].apply(get_date)


In [10]:
MESES = ["", "enero", "febrero", "marzo", "abril", "mayo", "junio", "julio", "agosto", "septiembre", "octubre", "noviembre", "diciembre"]

def get_name(date):
  mes = MESES[date.month]
  pattern = f"<tr>\s*<td><a[^>]*>([^<]+)</a></td>\s*<td> <a[^>]*>[^\d]+{date.day:02d} de {mes} {date.year}"
  result = re.findall(pattern, response.text)
  if len(result) == 0:
    pattern = f"<tr>\s*<td>([^<]+)</td>\s*<td>[^\d]+{date.day:02d} de {mes} {date.year}"
    result = re.findall(pattern, response.text)
  if len(result) == 0:
    print(date)
    return None
  return result[0]

df["motivo"] = df["fecha"].apply(get_name)

In [11]:
df[["fecha", "tipo", "motivo"]].to_json(f"feriados_{year}.json"
,orient="records"
,date_format="iso"
,force_ascii=False
,indent=2
)

In [12]:
df["tipo"]

0      Feriados inamovibles
1      Feriados inamovibles
2      Feriados inamovibles
3      Feriados inamovibles
4        Días no laborables
5      Feriados inamovibles
6       Feriados Turísticos
7      Feriados inamovibles
8        Días no laborables
9        Días no laborables
10     Feriados inamovibles
11     Feriados inamovibles
12    Feriados trasladables
13     Feriados inamovibles
14       Días no laborables
15     Feriados inamovibles
16    Feriados trasladables
17       Días no laborables
18       Días no laborables
19    Feriados trasladables
20    Feriados trasladables
21     Feriados inamovibles
22     Feriados inamovibles
Name: tipo, dtype: object